In [4]:
from plantbrain_fastml.models.regressors.linear_regression import LinearRegressionRegressor
from plantbrain_fastml.managers.regressor_manager import RegressorManager
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

# Load California housing dataset
data = fetch_california_housing()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, random_state=42)

# Initialize manager and add model
manager = RegressorManager()
model = LinearRegressionRegressor()
manager.add_model('LinearRegression', model)
from plantbrain_fastml.models.regressors.random_forest import RandomForestRegressorModel

model2 = RandomForestRegressorModel()  # NOT sklearn's RandomForestRegressor()
manager.add_model('RandomForest', model2)


# IMPORTANT: evaluate_all uses X, y for training + CV/hypertune
# Use entire training data for this; do not pass test set here
results = manager.evaluate_all(
    X_train, y_train,
    hypertune=True,
    hypertune_params={'n_trials': 10},
    cv_folds=5,
    return_hypertune_params=True,
    feature_elimination=True,
    fe_n_features=5,
    fe_method='lasso'
)

print(results)

# If you want to evaluate on test set separately:
test_scores = model.evaluate(X_test, y_test)
print("\nTest set evaluation:")
print(test_scores)


[I 2025-05-21 19:31:13,627] A new study created in memory with name: no-name-7135e5fd-89bd-4283-9c5e-bf9cf510ea23


[I 2025-05-21 19:31:16,994] Trial 0 finished with value: 0.723842683007923 and parameters: {'fit_intercept': True}. Best is trial 0 with value: 0.723842683007923.
[I 2025-05-21 19:31:18,961] Trial 1 finished with value: 0.723842683007923 and parameters: {'fit_intercept': True}. Best is trial 0 with value: 0.723842683007923.
[I 2025-05-21 19:31:20,771] Trial 2 finished with value: 0.723842683007923 and parameters: {'fit_intercept': True}. Best is trial 0 with value: 0.723842683007923.
[I 2025-05-21 19:31:20,813] Trial 3 finished with value: 0.723842683007923 and parameters: {'fit_intercept': True}. Best is trial 0 with value: 0.723842683007923.
[I 2025-05-21 19:31:20,849] Trial 4 finished with value: 0.7802127590785453 and parameters: {'fit_intercept': False}. Best is trial 0 with value: 0.723842683007923.
[I 2025-05-21 19:31:20,876] Trial 5 finished with value: 0.7802127590785453 and parameters: {'fit_intercept': False}. Best is trial 0 with value: 0.723842683007923.
[I 2025-05-21 19:3

[LinearRegression] Best params: {'fit_intercept': True}


[I 2025-05-21 19:31:44,869] Trial 0 finished with value: 0.5189885803403937 and parameters: {'n_estimators': 207, 'max_depth': 15, 'min_samples_split': 3, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.5189885803403937.
[I 2025-05-21 19:32:17,177] Trial 1 finished with value: 0.5125976108775561 and parameters: {'n_estimators': 245, 'max_depth': 19, 'min_samples_split': 2, 'min_samples_leaf': 2}. Best is trial 1 with value: 0.5125976108775561.
[I 2025-05-21 19:32:32,144] Trial 2 finished with value: 0.6684296815248846 and parameters: {'n_estimators': 296, 'max_depth': 5, 'min_samples_split': 6, 'min_samples_leaf': 1}. Best is trial 1 with value: 0.5125976108775561.
[I 2025-05-21 19:32:49,590] Trial 3 finished with value: 0.5406331046972129 and parameters: {'n_estimators': 194, 'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 4}. Best is trial 1 with value: 0.5125976108775561.
[I 2025-05-21 19:32:58,455] Trial 4 finished with value: 0.6686829441605248 and parameters: {

[RandomForest] Best params: {'n_estimators': 245, 'max_depth': 19, 'min_samples_split': 2, 'min_samples_leaf': 2}
                      rmse       mae        r2
model                                         
LinearRegression  0.723843  0.530441  0.607160
RandomForest      0.512796  0.335957  0.802788

Test set evaluation:
{'rmse': np.float64(0.7351215581896435), 'mae': np.float64(0.5249196585679878), 'r2': np.float64(0.5892155340782328)}


In [2]:
results

,rmse,mae,r2
model,,,
LinearRegression,0.723843,0.530441,0.607160
RandomForest,0.513200,0.336102,0.802483


In [7]:
model2.selected_features

[0, 2, 3, 6, 7]

In [3]:
manager.hypertune_params

{'LinearRegression': {'fit_intercept': True},
 'RandomForest': {'n_estimators': 188,
  'max_depth': 20,
  'min_samples_split': 3,
  'min_samples_leaf': 2}}